In [6]:
#Librerias
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
path ='/media/anmrodriguezsa/DA4E639F4E6372E5/Users/Ángela/Desktop/Proyecto1/acciones'  ##Linux
#path ='C:/Users/Ángela/Desktop/Proyecto1/acciones'   #windows

In [8]:
# Lectura de archivos .csv y Selección de los precios de cierre de cada acción.

def LecYSel(path):
    allFiles = glob.glob(path + "/*.csv")
    Pc=np.asarray([])
    flagFirst = 0
        
    for file_ in allFiles:
        Data = pd.read_csv(file_,index_col=None, header=0)  #Lectura de archivos .csv
        x  = Data["Último"].values   #Selección de los valores del precio de cierre de las acciones
        x = [s. replace('.','') for s in x] 
        x = [s. replace(',','.') for s in x]
        v = [float(s) for s in x] 
        
        if flagFirst==0:
            flagFirst = 1
            Pc = v
        else:
            Pc = np.vstack((Pc,v)) #"pega"
    return(Pc)

In [9]:
#Cálculo precios relativos

def retorn(M):
    M=np.transpose(M)
    vt=np.zeros((M.shape[0], M.shape[1]))
            
    for i in range(M.shape[1]):
        for j in range(M.shape[0]):
            vt[j][i] = M[j-1][i] /M[j][i] #Cálculo de vt (según artículo guía)
    
    Vt=np.transpose(np.delete(vt,0,0))
    onesColumn = np.ones((1,Vt.shape[1]));
    Vt = np.vstack((onesColumn,Vt)) # ubica elementos en la parte inferior - en este caso ubica filas-
    #plt.matshow(V_t)
    return Vt

In [10]:
# Insertar matriz R

Vt=np.matrix(retorn(LecYSel(path)))

R = Vt

# Q matrix
Q = np.matrix(np.zeros([R.shape[0],R.shape[1]]))

# Gamma (learning parameter).
gamma = 0.8

# Initial state. (Usually to be chosen at random)
initial_state = 1

# This function returns all available actions in the state given as an argument
def available_actions(state):
    current_state_row = R[state,]
    av_act = np.where(current_state_row >= 0)[1]
    return av_act


# Get available actions in the current state
available_act = available_actions(initial_state) 

# This function chooses at random which action to be performed within the range 

# of all the available actions.

def sample_next_action(available_actions_range):

    next_action = int(np.random.choice(available_act,1))

    return next_action



# Sample next action to be performed

action = sample_next_action(available_act)



# This function updates the Q matrix according to the path selected and the Q 

# learning algorithm

def update(current_state, action, gamma):

    

    max_index = np.where(Q[action,] == np.max(Q[action,]))[1]



    if max_index.shape[0] > 1:

        max_index = int(np.random.choice(max_index, size = 1))

    else:

        max_index = int(max_index)

    max_value = Q[action, max_index]

    

    # Q learning formula

    Q[current_state, action] = R[current_state, action] + gamma * max_value



# Update Q matrix

update(initial_state,action,gamma)



#-------------------------------------------------------------------------------

# Training



# Train over 10 000 iterations. (Re-iterate the process above).

for i in range(10000):

    current_state = np.random.randint(0, int(Q.shape[0]))

    available_act = available_actions(current_state)

    action = sample_next_action(available_act)

    update(current_state,action,gamma)

    

# Normalize the "trained" Q matrix

print("Trained Q matrix:")

print(Q/np.max(Q)*100)



#-------------------------------------------------------------------------------

# Testing

# Goal state = 5

# Best sequence path starting from 2 -> 2, 3, 1, 5

current_state = 2

steps = [current_state]

while current_state != 5:



    next_step_index = np.where(Q[current_state,] == np.max(Q[current_state,]))[1]

    

    if next_step_index.shape[0] > 1:

        next_step_index = int(np.random.choice(next_step_index, size = 1))

    else:

        next_step_index = int(next_step_index)

    

    steps.append(next_step_index)

    current_state = next_step_index



# Print selected sequence of steps

print("Selected path:")

print(steps)

Trained Q matrix:
[[ 98.98601993  99.43667774  99.32714844  99.23088773  99.08506033
   99.08473593  99.24033804  99.13692836  99.10063637  99.10216923
   99.00794085  99.12212803  99.23724852  99.18808916  99.36998027
   99.11230352  99.1628522   99.0135768   99.07085012]
 [ 99.02795451  99.47870299 100.          99.14351845  98.99867509
   99.31417395  99.35283331  99.19013329  99.02385492  99.04390328
   99.09579024  99.2325637   99.02966174  99.35336704  99.14131347
   99.08436223  99.34974334  99.10192534  99.30816897]
 [ 99.06455196  99.37795666  99.74707666  98.9629183   99.40599618
   99.47346949  99.24033804  98.81499581  99.22211561  98.76396706
   98.85007951  99.12212803  99.51748898  99.20812698  99.86308837
   98.72961246  98.98325251  98.91430878  99.24489629]
 [ 98.98601993  98.80334779  99.63778027  99.35348903  99.08506033
   99.63613105  99.42851251  98.42201608  99.44163072  99.05680942
   98.71750774  99.53276207  98.9451366   99.74276249  99.46286761
   99.1123035